# 🦌 Elk Model Explorer ⚡

The [Elk Model Repositoy](https://github.com/eclipse/elk-models) was converted to elk json and included as a set of fixtures to test against. This notebook loads those fixtures for users to explore to understand what kinds of layout variety possible as well and see the originating elk json.

In [ ]:
import json

from pathlib import Path
from IPython.display import display, JSON
from ipyelk import ElkDiagram, tests

In [ ]:
def fixture_explorer():
    #TODO some labels do not have a width / height provided ... could use the text sizer widget and size labels accordingly
    #TODO some nodes do not have a specified width / height ... set a default
    fixtures = Path(tests.__file__).parent / "fixtures"
    fixtures

    options = [f.relative_to(fixtures) for f in fixtures.rglob("*.json")]
    len(options)
    import ipywidgets as W
    diagram = ElkDiagram(layout={"flex":"1"})
    selector = W.Dropdown(options=options)
    err_btn = W.Button()
    output = W.Output()

    def set_err(message):
        if message:
            err_btn.icon = "warning"
            err_btn.tooltip = message
        else:
            err_btn.icon = "check"
            err_btn.tooltip = ""

    def _load(change=None):
        data = json.loads((fixtures / selector.value).read_text())
        try:
            output.clear_output()
            with output:
                display(JSON(data))
            diagram.value = {"id":"root"}  # needed for now to clear sprotty diagram. until fixed https://github.com/jupyrdf/ipyelk/issues/17
            diagram.value = data
            set_err("")
        except Exception as e:
            diagram.value = {"id":"root"}
            set_err(str(e))


    model_explorer = W.VBox(children=[

        W.HBox(
            children=[
                selector,
                err_btn,
            ]
        ),
        W.HBox(
            children=[
                diagram,
                output,
            ],
            layout={"flex":"1"})

    ],
        layout={"height":"100%"}  
    )
    selector.observe(_load, "value")
    _load()
    return model_explorer

if __name__ == "__main__":
    explorer = fixture_explorer()
    display(explorer)